# Ella Heath - Project Proposal Web Scraping

# To be used for initial scraping 

In [32]:
import pandas as pd 
from selenium import webdriver  # For controlling the web browser and interacting with HTML content
from selenium.webdriver.chrome.service import Service  # For setting up ChromeDriver as a service
from selenium.webdriver.common.by import By  # To locate HTML elements by XPath
from selenium.webdriver.chrome.options import Options  # For handling Chrome options
from webdriver_manager.chrome import ChromeDriverManager  # For automatic ChromeDriver installation
import time  # For handling wait times
import random 

# Scraping Source 

This data source has information about games results for only seeds and is not team specific. I can scrape this site to pull the table to see the best performance each seed has as well as their win-loss percentage. I will be able to visualize this data and show what seeds win the most and those who lose the most. 

In [33]:
#Set up ChromeDriver using webdriver_manager
chrome_options = Options() 
service = Service(ChromeDriverManager().install())  

#Create a ChromeDriver instance
browser = webdriver.Chrome(service=service, options=chrome_options)

#Open the website
url = 'https://www.ncaa.com/news/basketball-men/article/2025-02-05/records-every-seed-march-madness-1985-2024'
browser.get(url)

table1 = browser.find_element(By.TAG_NAME, "table")
headers1 = [header.text for header in table1.find_elements(By.TAG_NAME, "th")]
rows1 = table1.find_elements(By.TAG_NAME, "tr")

seed_data = []
for row in rows1[1:]:  
    cols = row.find_elements(By.TAG_NAME, "td")
    seed_data.append([col.text for col in cols])

seeds_df = pd.DataFrame(seed_data, columns=headers1)
display(seeds_df)

,SEED,OVERALL RECORD,BEST FINISH
0,No. 1,534-134,Champions (26 times)
1,No. 2,373-155,Champions (five times)
2,No. 3,294-156,Champions (four times)
3,No. 4,250-158,Champion (two times)
4,No. 5,183-160,Runner-up (four times)
5,No. 6,168-159,Champions (once)
6,No. 7,141-159*,Champions (once)
7,No. 8,113-159,Champions (once)
8,No. 9,98-160,Final Four (two times)
9,No. 10,97-159*,Final Four (once)


In [34]:
## Data Cleaning 
seeds_df['OVERALL RECORD']  = seeds_df['OVERALL RECORD'].str.replace("*","")
display(seeds_df)
seeds_df.to_csv("seed_records.csv", index=False)

## For "Best Finish" I would like to figure out how separate the values and make this column into two columns 
## and make the second columns value numeric. 

,SEED,OVERALL RECORD,BEST FINISH
0,No. 1,534-134,Champions (26 times)
1,No. 2,373-155,Champions (five times)
2,No. 3,294-156,Champions (four times)
3,No. 4,250-158,Champion (two times)
4,No. 5,183-160,Runner-up (four times)
5,No. 6,168-159,Champions (once)
6,No. 7,141-159,Champions (once)
7,No. 8,113-159,Champions (once)
8,No. 9,98-160,Final Four (two times)
9,No. 10,97-159,Final Four (once)


## Get the second table on the website 

In [35]:
# Locate the second table
table2 = browser.find_elements(By.TAG_NAME, "table")[1]

# Extract headers
headers2 = [header.text for header in table2.find_elements(By.TAG_NAME, "th")]

# Extract rows
rows2 = table2.find_elements(By.TAG_NAME, "tr")

# Extract data
seed_pct_data = []
for row in rows2[1:]:  
    cols = row.find_elements(By.TAG_NAME, "td")
    seed_pct_data.append([col.text for col in cols])

# Convert to DataFrame
seed_pct_df = pd.DataFrame(seed_pct_data, columns=headers2)

# Save or display
seed_pct_df.to_csv("seed_pct_records.csv", index=False)
display(seed_pct_df)

,SEED VS. SEED.,W-L,PCT.
0,1 vs. 16,158-2,.988
1,2 vs. 15,149-11,.931
2,3 vs. 14,137-23,.856
3,4 vs. 13,127-33,.794
4,5 vs. 12,103-57,.644
5,6 vs. 11,98-62,.613
6,7 vs. 10*,97-62,.610
7,8 vs. 9,77-83,.481


In [36]:
games_df = pd.read_csv('Project Proposal - Combined Games.csv')
games_df

,year,round_of,winning_team_name,winning_team_seed,winning_team_score,losing_team_name,losing_team_seed,losing_team_score
0,2024,64,UConn,1,91,Stetson,16,52
1,2024,64,Northwestern,9,77,Florida Atlantic,8,65
2,2024,64,San Diego State,5,69,UAB,12,65
3,2024,64,Yale,13,78,Auburn,4,76
4,2024,64,Duquesne,11,71,BYU,6,67
...,...,...,...,...,...,...,...,...
2452,1985,16,Memphis State#,2,59,Boston College,11,57
2453,1985,8,Memphis State#,2,63,Oklahoma,1,61
2454,1985,4,Georgetown,1,77,St Johns,1,59
2455,1985,4,Villanova,8,52,Memphis St. #,2,45


### Upset DataFrame

In [37]:
# Make sure seed columns are numeric
games_df['winning_team_seed'] = pd.to_numeric(games_df['winning_team_seed'], errors='coerce')
games_df['losing_team_seed'] = pd.to_numeric(games_df['losing_team_seed'], errors='coerce')

# Create a new DataFrame with only upsets
upsets_df = games_df[games_df['winning_team_seed'] > games_df['losing_team_seed']].copy()

display(upsets_df)
upsets_df.to_csv('combined_upsets.csv', index=False)

,year,round_of,winning_team_name,winning_team_seed,winning_team_score,losing_team_name,losing_team_seed,losing_team_score
1,2024,64,Northwestern,9,77,Florida Atlantic,8,65
3,2024,64,Yale,13,78,Auburn,4,76
4,2024,64,Duquesne,11,71,BYU,6,67
13,2024,16,Illinois,3,72,Iowa State,2,69
16,2024,64,Michigan State,9,69,Mississippi State,8,51
...,...,...,...,...,...,...,...,...
2448,1985,32,Louisiana Tech,5,79,Ohio State,4,67
2449,1985,32,Boston College,11,74,Duke,3,73
2453,1985,8,Memphis State#,2,63,Oklahoma,1,61
2455,1985,4,Villanova,8,52,Memphis St. #,2,45
